In [14]:
import csv
import math
import pandas as pd
from datetime import datetime

In [15]:
taxi16_df = pd.read_csv("../dataset_4_02_modified/13taxi_with_weather_std.csv", sep = ",")

In [16]:
def lag_timestamp(timestamp, lag):
    return (pd.to_datetime(timestamp) - pd.Timedelta(hours=lag)).strftime('%Y/%m/%d %H')

In [17]:
taxi16_df["Lag1 Timestamp"] = (pd.to_datetime(taxi16_df["Trip Start Timestamp"])\
                               - pd.Timedelta(hours=1)).dt.strftime('%Y/%m/%d %H')
taxi16_df["Lag2 Timestamp"] = (pd.to_datetime(taxi16_df["Trip Start Timestamp"])\
                               - pd.Timedelta(hours=2)).dt.strftime('%Y/%m/%d %H')
taxi16_df["Lag3 Timestamp"] = (pd.to_datetime(taxi16_df["Trip Start Timestamp"])\
                               - pd.Timedelta(hours=3)).dt.strftime('%Y/%m/%d %H')

In [18]:
taxi16_lean_df = taxi16_df[["Trip Start Timestamp", "Pickup Community Area", "Count"]]

In [19]:
# the data entries deleted because of lack of lag information are
# timestamps 2016/01/01 00, 2016/01/01 01, 2016/01/01 02, 
# 2016/03/13 03, 2016/03/13 04, 2016/03/13 05 because of the
# strange missing information entry at 2016/03/13 02
lag1_added_df = taxi16_df.merge(right = taxi16_lean_df, left_on = ["Lag1 Timestamp", "Pickup Community Area"],\
               right_on = ["Trip Start Timestamp", "Pickup Community Area"], how = "inner")\
                .drop(columns = ["Lag1 Timestamp", "Trip Start Timestamp_y"])\
                .rename(columns = {"Trip Start Timestamp_x": "Trip Start Timestamp",\
                                   "Count_x": "Count", "Count_y": "lag1"})

lag2_added_df = lag1_added_df.merge(right = taxi16_lean_df, left_on = ["Lag2 Timestamp", "Pickup Community Area"],\
               right_on = ["Trip Start Timestamp", "Pickup Community Area"], how = "inner")\
                .drop(columns = ["Lag2 Timestamp", "Trip Start Timestamp_y"])\
                .rename(columns = {"Trip Start Timestamp_x": "Trip Start Timestamp",\
                                   "Count_x": "Count", "Count_y": "lag2"})
        
lag3_added_df = lag2_added_df.merge(right = taxi16_lean_df, left_on = ["Lag3 Timestamp", "Pickup Community Area"],\
               right_on = ["Trip Start Timestamp", "Pickup Community Area"], how = "inner")\
                .drop(columns = ["Lag3 Timestamp", "Trip Start Timestamp_y"])\
                .rename(columns = {"Trip Start Timestamp_x": "Trip Start Timestamp",\
                                   "Count_x": "Count", "Count_y": "lag3"})

In [20]:
lag3_added_df.to_csv("2013lag.csv", index = False)

In [21]:
set(taxi16_lean_df["Trip Start Timestamp"].unique()) - set(lag3_added_df["Trip Start Timestamp"].unique())

{'2013/01/01 00',
 '2013/01/01 01',
 '2013/01/01 02',
 '2013/03/10 03',
 '2013/03/10 04',
 '2013/03/10 05'}

In [23]:
sum(taxi16_lean_df["Trip Start Timestamp"] == '2013/03/10 03')

77